In [14]:
import rpy2
import rpy2.robjects as robjects
r = robjects.r
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import statsmodels.api as sm
import pandas as pd
from itertools import combinations
import numpy as np

# Download Carlson Data

In [2]:
base = importr('base')
utils = importr('utils')
r('install.packages("FindIt", repos="https://CRAN.R-project.org/")')
findit = importr('FindIt')
'''using data from example in FindIt'''
robjects.r('data("Carlson", package = "FindIt")')
carlson = robjects.r['Carlson']
carlson_df = pandas2ri.rpy2py(carlson)
print(carlson_df)

'''trying a couple variables to use the 2 way amie'''
new_Record = robjects.r('''
Carlson$newRecordF<- factor(Carlson$newRecordF,ordered=TRUE,
levels=c("YesLC", "YesDis","YesMP",
"noLC","noDis","noMP","noBusi"))''')
promise = robjects.r('''
Carlson$promise <- factor(Carlson$promise,ordered=TRUE,levels=c("jobs","clinic","education"))
''')
model = robjects.r('''

fit2 <- CausalANOVA(formula=won ~ newRecordF + promise,
int2.formula = ~ newRecordF:promise,
data=Carlson, pair.id=Carlson$contestresp,diff=TRUE,
cluster=Carlson$respcodeS, nway=2)
''')

R[write to console]: trying URL 'https://CRAN.R-project.org/bin/macosx/big-sur-arm64/contrib/4.3/FindIt_1.2.0.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 430138 bytes (420 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console


The downloaded binary packages are in
	/var/folders/x2/9ng2kgzn3_163q6jl10rdmkr0000gn/T//RtmpwUf5No/downloaded_packages
      won  newRecordF  promise  coeth_voting  relevantdegree  respcodeS   
1       1           6        1             2               1          1  \
2       0           7        3             1               2          1   
3       1           7        3             1               2          1   
4       0           3        2             1               2          1   
5       0           7        1             1               1          1   
...   ...         ...      ...           ...             ...        ...   
3228    1           4        1             1               2        544   
3229    1           5        1             2               2        544   
3230    0           2        2             1               1        544   
3231    0           7        1             2               2        544   
3232    1           3        1             1          

In [6]:
#One Hot Encode
enc = OneHotEncoder(handle_unknown='ignore')
X = carlson_df.iloc[:,1:-2]
y = carlson_df.iloc[:,0].to_numpy() *2 - 1
trans_x = enc.fit_transform(X).toarray() *2 - 1
df =pd.DataFrame(trans_x)
print(df)
#0-6 are record, 7-9 are promise, 10-11 are coeth, 12-13 is degree
record_list = [i for i in range(7)]
promise_list = [7,8,9]
coeth_list = [10,11]
degree_list = [12,13]
for record in record_list:
    for promise in promise_list:
        df[f"{record}*{promise}"] = df[record] * df[promise]
        for coeth in coeth_list:
            df[f"{record}*{promise}*{coeth}"] = df[record] * df[promise] * df[coeth]
            for degree in degree_list:
                df[f"{record}*{promise}*{coeth}*{degree}"] = df[record] * df[promise] * df[degree] * df[coeth]
        for degree in degree_list:
                df[f"{record}*{promise}*{degree}"] = df[record] * df[promise] * df[degree]
    for coeth in coeth_list:
        df[f"{record}*{coeth}"] = df[record] * df[coeth]
        for degree in degree_list:
             df[f"{record}*{coeth}*{degree}"] = df[record] * df[coeth] * df[degree]
    for degree in degree_list:
        df[f"{record}*{degree}"] = df[record] * df[degree]

print(len(df.columns))

       0    1    2    3    4    5    6    7    8    9    10   11   12   13
0    -1.0 -1.0 -1.0 -1.0 -1.0  1.0 -1.0  1.0 -1.0 -1.0 -1.0  1.0  1.0 -1.0
1    -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  1.0 -1.0 -1.0  1.0  1.0 -1.0 -1.0  1.0
2    -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  1.0 -1.0 -1.0  1.0  1.0 -1.0 -1.0  1.0
3    -1.0 -1.0  1.0 -1.0 -1.0 -1.0 -1.0 -1.0  1.0 -1.0  1.0 -1.0 -1.0  1.0
4    -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  1.0  1.0 -1.0 -1.0  1.0 -1.0  1.0 -1.0
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
3227 -1.0 -1.0 -1.0  1.0 -1.0 -1.0 -1.0  1.0 -1.0 -1.0  1.0 -1.0 -1.0  1.0
3228 -1.0 -1.0 -1.0 -1.0  1.0 -1.0 -1.0  1.0 -1.0 -1.0 -1.0  1.0 -1.0  1.0
3229 -1.0  1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  1.0 -1.0  1.0 -1.0  1.0 -1.0
3230 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  1.0  1.0 -1.0 -1.0 -1.0  1.0 -1.0  1.0
3231 -1.0 -1.0  1.0 -1.0 -1.0 -1.0 -1.0  1.0 -1.0 -1.0  1.0 -1.0  1.0 -1.0

[3232 rows x 14 columns]
259


/var/folders/x2/9ng2kgzn3_163q6jl10rdmkr0000gn/T/ipykernel_61654/631177310.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{record}*{coeth}*{degree}"] = df[record] * df[coeth] * df[degree]
/var/folders/x2/9ng2kgzn3_163q6jl10rdmkr0000gn/T/ipykernel_61654/631177310.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{record}*{coeth}"] = df[record] * df[coeth]
/var/folders/x2/9ng2kgzn3_163q6jl10rdmkr0000gn/T/ipykernel_61654/631177310.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the res

In [72]:
# train test split
column_names = ['bias']
for column in df.columns:
    column_names.append(column)

T = df.to_numpy()
T = np.insert(T, 0, 1, axis=1)
T_train, T_test, y_train, y_test = train_test_split(T, y, test_size=0.1)
print(T_train)
print(y_train)

[[ 1. -1.  1. ... -1.  1. -1.]
 [ 1.  1. -1. ...  1. -1.  1.]
 [ 1. -1. -1. ...  1.  1. -1.]
 ...
 [ 1. -1. -1. ... -1. -1.  1.]
 [ 1. -1.  1. ...  1.  1. -1.]
 [ 1. -1.  1. ... -1. -1.  1.]]
[1 1 1 ... 1 1 1]


In [98]:


model2 = Lasso(alpha=0.01)
model2.fit(T_train, y_train)
y_pred = model2.predict(T_test)
# Retrieve the non-zero values using the indices
nonzero_indices = np.nonzero(model2.coef_)
print(nonzero_indices[0])
nonzero_values = model2.coef_[nonzero_indices]
count = y_test.shape[0]
correct = 0
for pred, act in zip(y_pred,y_test):
    if (pred >=0 and act ==1) or (pred <0 and act ==-1):
        correct+=1
print(correct/count)

[  1   2   4   7  15  22  25  26  28  29  30  31  32  42  45  66  67  69
  70  71  72  73  74  77  80  92  93  96  97 101 104 107 112 115 120 121
 122 123 124 127 128 139 145 146 147 148 149 150 157 158 160 161 162 163
 166 167 169 170 174 175 176 177 178 179 180 181 183 184 190 206 207 208
 217 220 226 227 228 229 230 231 236 237 243 245 246 248 249 250 251 253
 254 256 258 259]
0.5679012345679012


In [99]:
selected_column_names = [column_names[i] for i in nonzero_indices[0]]

for name, coef in zip(selected_column_names, model2.coef_[nonzero_indices[0]]):
    if abs(coef) < 0.01:
        continue
    print(name, coef)

0 0.052200962864239205
1 0.027472162922066626
3 -0.06106108725095298
6 -0.06909490310820536
0*8*10 0.010003641551980368
0*8*10*12 0.05541841082150994
0*8*12 0.02256407142549437
0*10 -0.021813581347311435
1*9*10 0.01786305030429376
1*9*10*12 0.021062909207003058
2*7*12 -0.020323170052227474
2*8*12 0.020902780609885745
2*9*10 0.0328921868115448
3*7*10 -0.02470109980852007
3*7*10*12 0.026990707519751637
3*7*12 -0.011477230217834278
3*9*12 0.015528595348561375
3*10 0.03181749921662195
4*7*10*12 0.029015421517910686
4*8*10*12 -0.024142567736732623
4*9*10 -0.02514546199957526
4*9*12 -0.023390986794159917
5*7 0.02378743082730197
5*8*12 -0.0306378186078069
5*9 -0.03112363803677283
5*10 0.062247063072847844
6*7*10 -0.014297908307334725
6*7*10*12 -0.010788207952696399
6*8*10*12 0.011148354964467849
6*9 0.010723242899110745
6*9*10*12 0.010111254386019486


In [75]:
print(column_names)

['bias', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '0*7', '0*7*10', '0*7*10*12', '0*7*10*13', '0*7*11', '0*7*11*12', '0*7*11*13', '0*7*12', '0*7*13', '0*8', '0*8*10', '0*8*10*12', '0*8*10*13', '0*8*11', '0*8*11*12', '0*8*11*13', '0*8*12', '0*8*13', '0*9', '0*9*10', '0*9*10*12', '0*9*10*13', '0*9*11', '0*9*11*12', '0*9*11*13', '0*9*12', '0*9*13', '0*10', '0*10*12', '0*10*13', '0*11', '0*11*12', '0*11*13', '0*12', '0*13', '1*7', '1*7*10', '1*7*10*12', '1*7*10*13', '1*7*11', '1*7*11*12', '1*7*11*13', '1*7*12', '1*7*13', '1*8', '1*8*10', '1*8*10*12', '1*8*10*13', '1*8*11', '1*8*11*12', '1*8*11*13', '1*8*12', '1*8*13', '1*9', '1*9*10', '1*9*10*12', '1*9*10*13', '1*9*11', '1*9*11*12', '1*9*11*13', '1*9*12', '1*9*13', '1*10', '1*10*12', '1*10*13', '1*11', '1*11*12', '1*11*13', '1*12', '1*13', '2*7', '2*7*10', '2*7*10*12', '2*7*10*13', '2*7*11', '2*7*11*12', '2*7*11*13', '2*7*12', '2*7*13', '2*8', '2*8*10', '2*8*10*12', '2*8*10*13', '2*8*11', '2*8*11*12', '2*8*1